# Import 

In [2]:

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten, MaxPool2D
from keras.optimizers import RMSprop
from keras.layers.advanced_activations import LeakyReLU, PReLU
from sklearn.cross_validation import StratifiedKFold
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import random as rn
import math
import matplotlib.pyplot as plt
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from numba import vectorize


Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [0]:
# random seed
seed = 43
np.random.seed(seed)

# Read in and preprocessing

In [0]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

In [6]:
test_data.shape

(28000, 784)

In [7]:
train_data_labels = train_data.label.values
train_data = train_data.drop(['label'], axis=1)
train_data[:5]

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
train_data = train_data.values.reshape(train_data.shape[0],28,28,1)
train_data = train_data.astype('float32')
train_data /= 255
len(train_data)

42000

In [0]:
train_data_labels = keras.utils.to_categorical(train_data_labels, 10)

In [0]:
test_data = test_data.values.reshape(test_data.shape[0],28,28,1)
X_test = test_data.astype('float32')
X_test /= 255

In [0]:
 X_train, X_val, Y_train, Y_val = train_test_split(train_data, train_data_labels, test_size=0.16, random_state=seed)

# Make Model

In [13]:

def createModel():
  model = Sequential()

  model.add(Conv2D(filters = 32, kernel_size = (5,5),
                   activation ='relu', input_shape = (28,28,1)))
  model.add(LeakyReLU(1/10))
  model.add(Conv2D(filters = 32, kernel_size = (5,5), 
                   activation ='relu'))
  model.add(LeakyReLU(1/10))
  model.add(Conv2D(filters = 32, kernel_size = (4,4), 
                   activation ='relu'))
  model.add(LeakyReLU(1/10))
  model.add(MaxPool2D(pool_size=(2,2)))



  model.add(Conv2D(filters = 64, kernel_size = (3,3), 
                   activation ='relu'))
  model.add(LeakyReLU(1/10))
  model.add(Conv2D(filters = 64, kernel_size = (3,3), 
                   activation ='relu'))
  model.add(LeakyReLU(1/10))
  model.add(Conv2D(filters = 64, kernel_size = (3,3), 
                   activation ='relu'))
  model.add(LeakyReLU(1/10))

  model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))



  model.add(Flatten())
  model.add(Dense(128, activation='relu'))

  model.add(Dense(64, activation='relu'))

  model.add(Dense(10, activation = "softmax"))

  model.compile(optimizer = 'adam' , loss = "categorical_crossentropy", metrics=["accuracy"])
  return model

model = createModel()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 24, 24, 32)        832       
_________________________________________________________________
leaky_re_lu_7 (LeakyReLU)    (None, 24, 24, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 20, 20, 32)        25632     
_________________________________________________________________
leaky_re_lu_8 (LeakyReLU)    (None, 20, 20, 32)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 17, 17, 32)        16416     
_________________________________________________________________
leaky_re_lu_9 (LeakyReLU)    (None, 17, 17, 32)        0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 8, 8, 32)          0         
__________

In [0]:

datagen = ImageDataGenerator(
        rotation_range=15,
        zoom_range = 0.15,
        width_shift_range=0.15,
        height_shift_range=0.15,
        horizontal_flip=False,
        vertical_flip=False)

In [0]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=6, 
                                            verbose=1, 
                                            factor=0.7, 
                                            min_lr=0.00001)

# Train Model
Should hit 0.99+ val_auc within 10-20 epochs. Running long enough will improve that farther. The model is pretty much immune to overfitting, and can still makes some small gains in the submission file after running it for a really long time (100+ epochs).

In [19]:
BATCH_SIZE = 64

model.fit_generator(datagen.flow(X_train, Y_train, batch_size= BATCH_SIZE),
                    epochs = 20, validation_data = (X_val, Y_val),
                    verbose = 2, steps_per_epoch=X_train.shape[0] // BATCH_SIZE
                    , callbacks=[learning_rate_reduction])



Epoch 1/20
 - 22s - loss: 0.0068 - acc: 0.9978 - val_loss: 0.0315 - val_acc: 0.9942
Epoch 2/20
 - 23s - loss: 0.0079 - acc: 0.9973 - val_loss: 0.0312 - val_acc: 0.9943
Epoch 3/20
 - 24s - loss: 0.0069 - acc: 0.9980 - val_loss: 0.0312 - val_acc: 0.9942
Epoch 4/20
 - 23s - loss: 0.0071 - acc: 0.9980 - val_loss: 0.0320 - val_acc: 0.9945
Epoch 5/20
 - 25s - loss: 0.0074 - acc: 0.9973 - val_loss: 0.0310 - val_acc: 0.9945
Epoch 6/20
 - 23s - loss: 0.0061 - acc: 0.9981 - val_loss: 0.0303 - val_acc: 0.9949
Epoch 7/20
 - 24s - loss: 0.0087 - acc: 0.9972 - val_loss: 0.0298 - val_acc: 0.9946
Epoch 8/20
 - 24s - loss: 0.0067 - acc: 0.9981 - val_loss: 0.0300 - val_acc: 0.9948
Epoch 9/20
 - 22s - loss: 0.0074 - acc: 0.9977 - val_loss: 0.0306 - val_acc: 0.9943
Epoch 10/20
 - 24s - loss: 0.0068 - acc: 0.9978 - val_loss: 0.0300 - val_acc: 0.9943
Epoch 11/20
 - 23s - loss: 0.0074 - acc: 0.9976 - val_loss: 0.0300 - val_acc: 0.9943
Epoch 12/20
 - 23s - loss: 0.0072 - acc: 0.9979 - val_loss: 0.0315 - val_a

# Save weights and make a submission

In [0]:
model.save_weights("Best_Weights.h5")

In [0]:
# make predictions
predictions = model.predict_classes(X_test, verbose=0)

submissions=pd.DataFrame({"ImageId": list(range(1,len(predictions)+1)),
                         "Label": predictions})
submissions.to_csv("MSNIST_Submission.csv", index=False, header=True)